In [1]:
# workaround via specifying an invalid value first
# %config Application.log_level='WORKAROUND'
# => fails, necessary on Fedora 27, ipython3 6.2.1
# %config Application.log_level='DEBUG'
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from polaris.anomaly import detector

from betsi import models, preprocessors, predictors
import json
from polaris.data.readers import read_polaris_data
from polaris.data import readers
from polaris.feature.cleaner import Cleaner
from polaris.feature.cleaner_configurator import CleanerConfigurator

In [2]:
# Plot configurations
plt.rcParams["figure.figsize"] = (25,8)

## Fetching/Loading Data with Polaris tools

In [3]:
# polaris fetch --start_date 2020-01-21 --end_date 2020-01-28 --cache_dir /tmp/LightSail-2 LightSail-2 /tmp/LightSail2-normalized_frames.json

In [4]:
path_to_data = "/home/ayush/dev/LightSail2-normalized_frames.json" # path to saved normalized frames

## Setting Configurations

In [5]:
layer_dims = [64,32]
stride=1
noise_margin_per = 50

In [6]:
# Read data from import file
metadata, data = readers.read_polaris_data(path_to_data)


## Cleaning Data

In [7]:
clean_params = CleanerConfigurator()
feature_cleaner = Cleaner(metadata, clean_params.get_configuration())
data = feature_cleaner.drop_constant_values(data)
data = feature_cleaner.drop_non_numeric_values(data)
data = feature_cleaner.handle_missing_values(data)
data = data.select_dtypes("number")

## Preprocess, train and predict data

In [8]:
normalizer, models, history, tt_data = detector.preprocess_train_model(
    data, layer_dims)

encoder_model = models[1]

# Get the bottleneck prediction
df_pred_bn = encoder_model.predict(tt_data[1])

events, distance_measure = detector.detect_events(df_pred_bn, noise_margin_per)

anomaly_metrics = {
    "history": history.history,
    "events": events,
}

Epoch 1/20
2/2 [==============================] - 1s 8ms/step - loss: 1.0072 - MSE: 1.0028
Epoch 2/20
2/2 [==============================] - 0s 11ms/step - loss: 0.9975 - MSE: 0.9933
Epoch 3/20
2/2 [==============================] - 0s 6ms/step - loss: 0.9859 - MSE: 0.9818
Epoch 4/20
2/2 [==============================] - 0s 7ms/step - loss: 0.9662 - MSE: 0.9619
Epoch 5/20
2/2 [==============================] - 0s 3ms/step - loss: 0.9548 - MSE: 0.9501
Epoch 6/20
2/2 [==============================] - 0s 5ms/step - loss: 0.9406 - MSE: 0.9355
Epoch 7/20
2/2 [==============================] - 0s 5ms/step - loss: 0.9214 - MSE: 0.9159
Epoch 8/20
2/2 [==============================] - 0s 5ms/step - loss: 0.9087 - MSE: 0.9029
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.8851 - MSE: 0.8790
Epoch 10/20
2/2 [==============================] - 0s 5ms/step - loss: 0.8728 - MSE: 0.8665
Epoch 11/20
2/2 [==============================] - 0s 3ms/step - loss: 0.8621 - MSE: 0.8

## Get Time indexes for test data

In [20]:
# tt_data, models, anomaly_metrics, events, df_pred_bn, distance_measure = res
time_index = pd.to_datetime(data.time, unit="s")
ndf = tt_data[1].to_numpy()
tt_data[1].isna().sum().sum()
ndf_time = (time_index.iloc[::stride]).iloc[:-1]
required_time = ndf_time[174:]
required_time = required_time.tolist()

required_time = [ str(x) for x in mango]

## Get Data in Required format

In [26]:
# plt.plot(mango[5])

final_output = []
for index,distance in enumerate(distance_measure):
    element = {}
    element["distance"] = distance
    element["time"] = required_time[index]
    element["event"] = index in events
    final_output.append(element)


final_output

[{'distance': 0.32863230837582447,
  'time': '2020-01-26 13:02:01',
  'event': False},
 {'distance': 0.4215626183332279,
  'time': '2020-01-26 13:02:08',
  'event': False},
 {'distance': 0.6065173296821329,
  'time': '2020-01-26 13:15:40',
  'event': False},
 {'distance': 0.621210674504742,
  'time': '2020-01-26 13:15:47',
  'event': False},
 {'distance': 0.5558589945403827,
  'time': '2020-01-26 14:45:58',
  'event': False},
 {'distance': 0.9430338662020799,
  'time': '2020-01-26 14:46:12',
  'event': False},
 {'distance': 0.37718104746287295,
  'time': '2020-01-26 14:46:33',
  'event': False},
 {'distance': 0.3235318823284014,
  'time': '2020-01-26 14:46:40',
  'event': False},
 {'distance': 0.41998038997632964,
  'time': '2020-01-26 14:46:47',
  'event': False},
 {'distance': 0.9554576052492539,
  'time': '2020-01-26 14:46:54',
  'event': False},
 {'distance': 1.3673178054172008,
  'time': '2020-01-26 14:47:01',
  'event': True},
 {'distance': 0.618406561822312,
  'time': '2020-01-2

## Output data to json file

In [28]:
with open('data.json', 'w') as outfile:
    json.dump(final_output, outfile)

# Use the output json data as input for graph generation in